In [1]:
import xarray as xr

In [2]:
cmems = xr.load_dataset("../1_1_Forcing/1_optional_extract_barents_sea/Barents_sea_forcing.zarr", engine="zarr")
cmems

<xarray.Dataset> Size: 351kB
Dimensions:              (depth: 1, latitude: 1, longitude: 1, time: 7305)
Coordinates:
  * depth                (depth) int64 8B 0
  * latitude             (latitude) int64 8B 72
  * longitude            (longitude) int64 8B 20
  * time                 (time) datetime64[ns] 58kB 1998-01-01 ... 2017-12-31
Data variables:
    T                    (depth, latitude, longitude, time) float64 58kB 5.20...
    npp                  (latitude, longitude, time) float64 58kB 3.654e-06 ....
    pelagic_layer_depth  (latitude, longitude, time) float64 58kB 300.7 ... 3...
    zeu                  (latitude, longitude, time) float64 58kB 100.2 ... 1...
    zooc                 (latitude, longitude, time) float64 58kB 0.8288 ... ...
Attributes:
    Conventions:  CF-1.7
    history:      Created on 2024-10-17
    institution:  CLS
    references:   http://www.cls.fr; http://www.seapodym.eu
    source:       SEAPODYM-LMTL 3.0.0
    title:        Global ocean low and mid trophic levels biomass content hin...

In [3]:
cafe = xr.open_dataset("../1_1_Forcing/raw/CAFE_NPP_8D_25KM_1998_2022.nc")
cafe = cafe.rename({"lat": "latitude", "lon": "longitude"})

In [4]:
cafe = cafe.sel(latitude=cmems.latitude, longitude=cmems.longitude, method="nearest")
cafe

<xarray.Dataset> Size: 18kB
Dimensions:    (time: 1150, latitude: 1, longitude: 1)
Coordinates:
  * longitude  (longitude) float32 4B 20.12
  * latitude   (latitude) float32 4B 72.12
  * time       (time) datetime64[ns] 9kB 1998-01-01 1998-01-09 ... 2022-12-27
Data variables:
    CAFE       (time, latitude, longitude) float64 9kB ...
Attributes:
    PI Name:      Thomas Ryan-Keogh
    PI Contact:   tryankeogh@csir.co.za
    History:      2023-01-07 08:10:43.118359 Python
    Conventions:  CF-1.8
    Keywords:     Primary Production

In [29]:
import plotly.express as px
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=cmems.time.data.flatten(),
        y=cmems.npp.data.flatten(),
        mode="lines",
        name="VGPM",
        line={"color": "red", "width": 1},
    )
)

fig.add_trace(
    go.Scatter(
        x=cafe.time.data.flatten(),
        y=cafe.CAFE.data.flatten(),
        mode="lines",
        name="CAFE",
        line={"color": "blue", "width": 1},
    )
)

fig.update_layout(
    title="Comparison of NPP between VGPM and CAFE in the Barents sea",
    xaxis_title="Time",
    yaxis_title="Primary production (mg C m-2 d-1)",
    legend_title="Dataset",
)

In [30]:
fig.write_html("npp_comparison_cafe_vgpm.html")